In [1]:
#this file is used to preprocess the training/test data images and is not used in the final app
import numpy as np
import cv2
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import pprint
from matplotlib import pyplot as plt
import glob
import shutil
import os
from face_toolbox_keras.models.detector import face_detector
np.set_printoptions(threshold=np.inf)
from face_toolbox_keras.models.parser import face_parser


In [2]:
def resize_image(im, max_size=450):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
    return im

In [8]:
src_dir = r"test_data/facesTestImages"
dst_dir = r"test_data/facesTestImagesProcessed"
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-4_keras.h5"
)
prs = face_parser.FaceParser()

for image in  glob.iglob(os.path.join(src_dir, "*.jpg")):
    imageName = image.rsplit('\\', 1)[1]
    currentImage = cv2.imread(image)[..., ::-1]
    resizedImage = resize_image(currentImage)
    bboxes = fd.detect_face(resizedImage, with_landmarks=False)
    assert len(bboxes) > 0, "No face detected."
    x0, y0, x1, y1, score = bboxes[0]
    x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
    face = resizedImage[x0:x1, y0:y1, :]
    mask = prs.parse_face(face)
    for i in range(len(mask)):
        for j in range(len(mask[0])):
            for k in range(len(mask[0][0])):
                if mask[i][j][k] != 1:
                    mask[i][j][k] = 0
    newmask = mask[0]
    newImg = cv2.bitwise_and(face,face,mask=newmask)
    resizedFace = cv2.resize(newImg, (250, 187), interpolation= cv2.INTER_LINEAR)
    os.chdir(dst_dir)
    cv2.imwrite( imageName, cv2.cvtColor(resizedFace, cv2.COLOR_RGB2BGR))

Resize image to (311, 450).
Resize image to (330, 450).
Resize image to (316, 450).
